In [ ]:
import sys, importlib
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.io as pio
pio.kaleido.scope.mathjax = None
import plot
from plot import (get_results_df,  group_results, get_best_results, 
                  calc_relative_lit_scores, calc_relative_stl_scores, calc_relative_xgboost_scores, 
                  plot_tasks, plot_summarize, plot_box, plot_box_binned, plot_scatter,
                  dict_mode, dict_mtl_mode, dict_resources, name_mapping,
                  merge_with_correlations, get_task_span, fit_power_law)
import cd_plot
from cd_plot import draw_cd_diagram

In [ ]:
PLOT_DIR = 'plots'

In [ ]:
if not os.path.exists(PLOT_DIR):
    os.makedirs(PLOT_DIR)

# load results
results = pd.read_pickle('results.pkl')
grouped = group_results(results, 'mode', add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])

#load correlation df
df_corr = pd.read_csv('correlation_results.csv')

In [ ]:
print('size results', results.shape)
print('size grouped', grouped.shape)

### Check for missing results

In [ ]:
# Check for missing szenarios
GROUPING = ['dataset', 'Task', 'resources', 'framework']
df = grouped.groupby(by=GROUPING, as_index=False).count()
df_test = df.loc[~((df['mode']==42) | ((df['mode']==30) & (df['dataset']=='xiong')))]
df_test = df_test.loc[~((df_test['mode']==7) & (df_test['framework']=='FFNN_input'))]
df_test = df_test.loc[~((df_test['mode']==5) & (df_test['framework']=='FFNN_input') & (df['dataset']=='xiong'))]
df_test = df_test.loc[~((df_test['mode'].isin([3,4])) & (df_test['framework']=='FFNN_stl'))]
df_test = df_test.loc[~((df_test['mode']==1) & (df_test['framework'].isin(['FFNN_mtl', 'MMOE', 'MTLNET'])))]
df_test

## Plots for MTL-Paper

## Fig: 6: Example Hu sparse-task 0.25

In [ ]:
TARGET = 'relative_error'
GROUP = 'mtl_mode' # mode, mtl_mode
#TASK_LIST = results['dataset-task-key'].unique()
TASK_LIST = ['hu_elongation', 'hu_yield-strength', 'hu_tensile-strength']
QUERY_PLOT = f"`best-of-{GROUP}` == 1 and `sparse_mode` == 'sparse-task' and sparse_amount == '0.25' and resources in ['15_min', '200_trails'] and mtl_mode in ['MTL-RC', 'STL']"
#QUERY_PLOT = f"framework == 'AutoSklearn' and mode in ['MTL-predict-all','STL'] and `sparse_mode` == 'sparse-task' and sparse_amount == '0.25' and resources in ['15_min', '200_trails'] and mtl_mode in ['MTL-RC', 'STL']"
df_plot = results.query(QUERY_PLOT)

fig = plot_tasks(df_plot,  tasks=TASK_LIST, target=TARGET, group=GROUP, plot_dict=eval(f'dict_{GROUP}'))
 
QUERY_PLOT = f"`dataset-task-key` in {TASK_LIST} and `sparse_mode` == 'sparse-task' and sparse_amount == '0.25' and mtl_mode == 'STL' and framework == 'XGBoost'"
df_add = results.query(QUERY_PLOT)

fig.add_trace(go.Box(
            y=df_add['dataset-task-key'],
            x=df_add[TARGET],
            name='STL - XGB',
            marker_color='#555555',
            # fillcolor=framework_dict['colour'],
            # line_color='black',
            line_width=2,
            orientation = 'h'
            #boxpoints='all'
        ))

desired_order = ['STL - XGB', 'STL', 'MTL-RC'] 
ordered_traces = [trace for group_name in desired_order for trace in fig.data if trace.name == group_name]
fig.data = ordered_traces
fig.update_layout(width=1100, height=400)
fig.update_layout(xaxis_range=[0.029, 0.101])
fig.write_image(os.path.join(PLOT_DIR, f'{TASK_LIST[0]}_sparse-task_0.25.svg'))
fig.show()

## Fig: 7, 8: summarized Results - combined mode and mtl-mode

In [ ]:
TARGET = 'relative_rmse_stl_median'
GROUP = 'mode' # mode, mtl_mode
PLOT = 'full' # Fig 6: 'full'; Fig 7: 'sparse'

if PLOT=='sparse':
    best_results = f"`best-of-{GROUP}` == 1 and resources == '15_min' and `sparse_mode` in ['sparse-all', 'sparse-task']  and mtl_mode in ['MTL-RC'] "
if PLOT=='full':
    best_results = f"`best-of-{GROUP}` == 1 and resources == '15_min' and `sparse_mode` == 'full' and mtl_mode in ['MTL-RC', 'MTL-NN'] "

best_of_results = results.query(best_results)
df_plot = group_results(best_of_results, GROUP, add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])
df_plot = df_plot.sort_values(by=['sparse_mode', 'sparse_amount', 'size', 'framework'], ascending=[True, False, False, True])

fig = plot_summarize(df_plot, group=GROUP, target=TARGET, plot_dict=eval(f'dict_{GROUP}'), save_fig=None)


# apend mtl-mode MTL-RC and MTL-NN
GROUP_ADD = 'mtl_mode'
if PLOT=='sparse':
    best_results = f"`best-of-{GROUP_ADD}` == 1 and resources == '15_min' and `sparse_mode` in ['sparse-all', 'sparse-task']  and mtl_mode in ['MTL-RC'] "
if PLOT=='full':
    best_results = f"`best-of-{GROUP_ADD}` == 1 and resources == '15_min' and `sparse_mode` == 'full' and mtl_mode in ['MTL-RC', 'MTL-NN'] "
best_of_results = results.query(best_results)
df_add = group_results(best_of_results, GROUP, add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])
df_add = df_add.sort_values(by=['sparse_mode', 'sparse_amount', 'size', 'framework'], ascending=[True, False, False, True])
df_add['sparse_key'] = df_add['sparse_mode'] + df_add['sparse_amount'].astype(str)

for group_, group_specs in eval(f'dict_{GROUP_ADD}').items():
    fig.add_trace(go.Box(
                y=df_add[TARGET].loc[df_add[GROUP_ADD] == group_],
                x=df_add['sparse_key'].loc[df_add[GROUP_ADD] == group_],
                name=group_,
                marker_color=group_specs['colour'],
                # fillcolor=framework_dict['colour'],
                # line_color='black',
                line_width=2,
                #boxpoints='all'
            ))

fig.update_traces(whiskerwidth=1, selector=dict(type='box'))
fig.update_traces(quartilemethod="exclusive") 


# Reihenfolge anpassen
desired_order = ['MTL-predict-other', 'MTL-predict-all', 
                 'MTL-predict-other-unc', 'MTL-predict-all-unc', 'MTL-true-other', 'MTL-RC',
                 'FFNN_mtl', 'FFNN_input', 'MTLNET', 'MMOE', 'MTL-NN']

# Reorder traces based on the desired order
ordered_traces = [trace for group_name in desired_order for trace in fig.data if trace.name == group_name]

# Update the fig data with the ordered traces
fig.data = ordered_traces


# Rename the traces according to the name mapping
for trace in ordered_traces:
    if trace.name in name_mapping:
        trace.name = name_mapping[trace.name]

if PLOT=='full':
    fig.update_layout(width=1200, height=500)
    fig.write_image(os.path.join(PLOT_DIR, f'plot_summarize_{GROUP}_full_combined.svg'))
if PLOT=='sparse':
    fig.update_layout(width=1200, height=625)
    fig.write_image(os.path.join(PLOT_DIR, f'plot_summarize_{GROUP}_sparse_combined.svg'))
fig.show()

## Fig 9: CD-Plots

In [ ]:
importlib.reload(sys.modules['cd_plot'])
import cd_plot
from cd_plot import draw_cd_diagram

TARGET = 'rmse'
GROUP = 'mode' # mode, mtl_mode

sparse_mode = 'sparse-task' # Fig 8a: 'full'; Fig 8b: 'sparse-all; Fig 8c: 'sparse-task''
sparse_amount = 'all'


QUERY_PLOT = f"`best-of-{GROUP}` == 1 and `sparse_mode` == '{sparse_mode}' and resources in ['15_min'] and `mode` in ['STL', 'MTL-true-other', 'MTL-predict-other', 'MTL-predict-all']"
#QUERY_PLOT = f"`best-of-{GROUP}` == 1 and `sparse_mode` == '{sparse_mode}' and resources in ['15_min'] and `mtl_mode` in ['STL', 'MTL-RC']"
df_plot = results.query(QUERY_PLOT)
print(df_plot.columns)
df_perf = df_plot[[GROUP, 'dataset-task-key', 'split', 'sparse_mode', 'sparse_amount', TARGET]].copy()
df_perf['task_key'] = df_perf['dataset-task-key'] + '_' + df_perf['split'].astype(str) + '_' + df_perf['sparse_mode'] + '_' + df_perf['sparse_amount'].astype(str)

df_perf = df_perf.rename(columns={GROUP: 'model_name',
                                  TARGET: 'metric'})
name_mapping = {
    'MTL-true-other': 'true-other',
    'MTL-predict-other': 'pred-other',
    'MTL-predict-all': 'pred-all',
    'MTL-predict-other-unc': 'pred-other-unc',
    'MTL-predict-all-unc': 'pred-all-unc',
    'MTL-RC': 'MTL-RC-combined',
    'MTL-NN': 'MTL-NN-combined'
}
df_perf['model_name'] = df_perf['model_name'].replace(name_mapping)
draw_cd_diagram(df_perf=df_perf, title='RMSE', labels=True, smaller_is_better=True, save_fig=os.path.join(PLOT_DIR, f'cd_plot_{GROUP}_{sparse_mode}_{sparse_amount}.pdf'))

In [ ]:
GROUP = 'mtl_mode'

best_results = f"`best-of-mtl_mode` == 1 and resources == '15_min' and mtl_mode in ['STL', 'MTL-RC', 'MTL-NN']"
df_plot = results.query(best_results).copy()
df_plot['shape-ratio'] = df_plot['shape-ratio'] * pd.to_numeric(df_plot['sparse_amount'])
GROUPING = ['shape-ratio', 'mtl_mode', 'resources']
df_plot = df_plot.groupby(by=GROUPING, as_index=False).median(numeric_only=True)
df_plot = df_plot[~df_plot['relative_error'].isna()]


fit_results = fit_power_law(df_plot, 'relative_error', GROUP)

## Fig 10: scaled error over Data-Shape

In [ ]:
SPARSE_MODE = 'sparse-all' # Fig 11 is combination of 'sparse-all' and 'sparse-task'
BINS = [0, 10, 20, 30, 100]
GROUP = 'mtl_mode'

color_dict = {'STL':  {'colour':'#57cca6'},
              'MTL-RC':{'colour':'#918DC3'},
              'MTL-NN':{'colour':'#00305e'}}

best_results = f"`best-of-mtl_mode` == 1 and `sparse_mode` =='{SPARSE_MODE}' and resources == '15_min' and mtl_mode in ['STL', 'MTL-RC', 'MTL-NN']"
df_plot = results.query(best_results).copy()
df_plot['shape-ratio'] = df_plot['shape-ratio'] * pd.to_numeric(df_plot['sparse_amount'])
GROUPING = ['shape-ratio', 'mtl_mode', 'resources']
df_plot = df_plot.groupby(by=GROUPING, as_index=False).median(numeric_only=True)

fig = plot_box_binned(df_plot, 'relative_error', 'shape-ratio', GROUP, custom_bins=BINS, color_dict=color_dict)
fig.update_layout(width=800, height=350)
fig.update_layout(yaxis = dict(tick0 = 0.01, dtick = 0.01))
fig.update_layout(yaxis_range=[0.0075,0.0975])
fig.write_image(os.path.join(PLOT_DIR, f"relative_error_over_datashape_binned_{SPARSE_MODE}.svg"))
fig.show()

## Fig 11a: Plot MTL Performance over STL Performance - same task

In [ ]:
GROUP = 'mtl_mode' # mode, mtl_mode
TARGET = 'relative_mape_stl_median'

best_results = f"`best-of-{GROUP}` == 1 and `sparse_mode` in ['sparse-task', 'sparse-all'] and resources in ['15_min'] and mtl_mode in ['MTL-RC', 'STL']"
best_of_results = results.query(best_results)
df_plot = group_results(best_of_results, GROUP, add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])

df_list = []
for group, group_df in df_plot.groupby(['dataset-task-key', 'sparse_mode', 'sparse_amount', 'resources']):
    get_best_stl = f"`mode` == 'STL'  and `best-of-mode` == 1"
    group_df[f'stl_mape'] = group_df.query(get_best_stl)['mape_median'].item()
    df_list.append(group_df)

df_plot = pd.concat(df_list)
df_plot = merge_with_correlations(df_plot, df_corr, aggregation_method='mean')
df_plot_same_task = df_plot.copy()

fig = plot_scatter(df_plot, TARGET, 'stl_mape', GROUP, z_color=df_plot['Spearman_aggregated'])
fig.update_layout(xaxis_range=[0,0.51])
fig.update_layout(width=800, height=480)
fig.update_traces(marker=dict(size=10))
fig.write_image(os.path.join(PLOT_DIR, f'plot_mtl_perfom_over_stl_perform.svg'))
fig.show()

## Fig 11b: Plot MTL Performance over STL Performance - other Task

In [ ]:
GROUP = 'mtl_mode' # mode, mtl_mode
TARGET = 'relative_mape_stl_median'

best_results = f"`best-of-{GROUP}` == 1 and `sparse_mode` in ['full', 'sparse-task', 'sparse-all'] and resources in ['15_min'] and mtl_mode in ['MTL-RC', 'STL'] "
best_of_results = results.query(best_results)
df_plot = group_results(best_of_results, GROUP, add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])

df_list = []
for group, group_df in df_plot.groupby(['dataset', 'sparse_mode', 'sparse_amount', 'resources']):
    for task in group_df.Task.unique():
        get_best_stl = f"`mode` == 'STL' and `best-of-mode` == 1 and dataset =='{group[0]}' and Task != '{task}' and resources in ['15_min']"
        if group[1]=='full':
            continue
        elif group[1]=='sparse-task':
            # The full results must be queried here for sparse-task
            sparse_mode_specific = f"sparse_mode =='full'"
        elif group[1]=='sparse-all':
            # The following must be queried from the sparse-all results for sparse-all
            sparse_mode_specific = f"sparse_mode =='sparse-all'"
        get_best_stl = get_best_stl + ' and ' + sparse_mode_specific
        stl_error = df_plot.query(get_best_stl)['mape_median'].max()
        task_df = group_df.loc[group_df['Task']==task].copy()
        task_df[f'stl_mape_other'] = stl_error
        df_list.append(task_df)


df_plot = pd.concat(df_list)
df_plot = merge_with_correlations(df_plot, df_corr, aggregation_method='mean')


try:
    df_plot_same_task
except NameError:
    print("`df_plot_same_task` from cell above needed!")
    
df_plot = df_plot.merge(df_plot_same_task[['dataset-task-key', 'sparse_mode', 'sparse_amount', 'stl_mape']], 
                        on=['dataset-task-key', 'sparse_mode', 'sparse_amount'], 
                        how='left')

fig = plot_scatter(df_plot, TARGET, 'stl_mape_other', GROUP, z_color=df_plot['stl_mape'])
fig.update_layout(xaxis_range=[0,0.51])
fig.update_layout(width=800, height=480)
fig.update_traces(marker=dict(size=10))
fig.write_image(os.path.join(PLOT_DIR, f'plot_mtl_perfom_over_stl-other_perform_.svg'))
fig.show()


## Fig 11: Compare to XGBoost Performance

In [ ]:
TARGET = 'relative_rmse_xgboost_median'
GROUP = 'mtl_mode' # mode, mtl_mode

results = calc_relative_xgboost_scores(results)

best_results = f"`best-of-{GROUP}` == 1 and mtl_mode in ['MTL-RC', 'STL']"
best_of_results = results.query(best_results)
df_plot = group_results(best_of_results, GROUP, add_const_num_cols=['best-of-mode', 'best-of-mtl_mode'])


fig = plot_box(df_plot, y_axis = GROUP, x_axis=TARGET, group='resources', target=TARGET, plot_dict=f'dict_resources')
fig.update_layout(width=1100, height=400)
fig.update_traces(orientation='h') 
fig.add_vline(x=1, line_color="black")
fig.update_layout(xaxis=dict(dtick=0.25))
fig.update_layout(xaxis_range=[0.7, 3.05])
fig.write_image(os.path.join(PLOT_DIR, f'plot_compare_time_to_xgboost.svg'))
fig.show()

# Additional plots

## Fig 10: R2 Values for Data-Shape and Data-Size

In [ ]:
GROUP = 'mtl_mode'

best_results = f"`best-of-mtl_mode` == 1 and resources == '15_min' and mtl_mode in ['STL', 'MTL-RC', 'MTL-NN']"
df_plot = results.query(best_results).copy()
df_plot['shape-ratio'] = df_plot['shape-ratio'] * pd.to_numeric(df_plot['sparse_amount'])
GROUPING = ['shape-ratio', 'mtl_mode', 'resources']
df_plot = df_plot.groupby(by=GROUPING, as_index=False).median(numeric_only=True)
df_plot = df_plot[~df_plot['relative_error'].isna()]


fit_results = fit_power_law(df_plot, 'relative_error', GROUP)

## Task specific plots

In [ ]:
TARGET = 'relative_score_stl'
GROUP = 'mtl_mode' # mode, mtl_mode
TASK_LIST = results['dataset-task-key'].unique()
QUERY_PLOT = f"`best-of-{GROUP}` == 1 and `sparse_mode` == 'sparse-all' and sparse_amount == '0.5' and resources in ['15_min', '200_trails']"
df_plot = results.query(QUERY_PLOT)

dict_task_mtl_mode = {'STL':  {'colour':'#57cca6', 'fill-colour':'#d6fff2'},
                      'MTL-RC':{'colour':'#00305e', 'fill-colour':'#b4cade'},
                      'MTL-NN':{'colour':'#403b7a', 'fill-colour':'#e3c6f7'}
                     }

fig = plot_tasks(df_plot, target=TARGET, group=GROUP, plot_dict=dict_task_mtl_mode)
fig.update_layout(width=1100, height=1100)
fig.show()